## Packages utilizados

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
path_confirmed = "../input/covid-19-cssegisanddata/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
path_deaths = "../input/covid-19-cssegisanddata/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
path_recovered = "../input/covid-19-cssegisanddata/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"

> Dataframe correspondente aos casos confirmados

In [ ]:
death_cases = pd.read_csv(path_deaths)
recovered_cases = pd.read_csv(path_recovered)

df = pd.read_csv(path_confirmed)
df.drop( labels = ["Province/State","Country/Region","Lat","Long"], inplace = True , axis = 1)
df

### Todo este processo é relativo à leitura do csv confirmed cases e algumas transformações de modo a que tenhamos o número confirmado de casos com Covid-19 por dia

   1.  Apenas precisamos dos casos confirmados
   2.  Apenas queremos o número de casos por dia
   3.  Tudo num numpy array em vez de um dataframe

In [ ]:
data = df.sum().to_frame(name='cases').to_numpy()
data

### Normalização dos dados através do min-max

> Scaler contém a normalização, a escala máxima e mínima e consegue desnormalizar

In [ ]:
from sklearn.preprocessing import MinMaxScaler

#fit and transform or fit_transform
scaler = MinMaxScaler(feature_range=(-1, 1))
data = scaler.fit_transform(data)


print("data normalizada = " + str(data.shape))

> Como podemos observar pelo o seguinte gráfico, o crescimento do Covid-19 é semelhante a um crescimento linear, e nos ultimos 10 dias, é semelhante a um crescimento exponencial, pelo que a previsão dos 7 dias seguintes deve ser parecido a um crescimento exponencial

In [ ]:
plt.plot(data)
plt.title("Covid-19")
plt.xlabel("Days")
plt.ylabel("Confirmed Cases")
plt.show()

## Esta função serve para preparar o dataset, respetivamente prepara os dados de treino e os respetivos labels da seguinte forma:

> Imaginemos x = [1,2,3,4,5,6] e um timestep de 2

> Então x = [1,2],[2,3].. e y = [3,4],[4,5]..


> Forecasting ou é single ou multi, contendo impacto nos labels, caso seja multi y =[3,4], caso seja single y = [3]

In [ ]:
def makeXy(data,time_step,forecasting):
    
    X = []
    Y = []
    dataset = []
    
    i = 0
    step = time_step
    step_y = time_step
    
    if(forecasting == "single"):
        step_y = 1
        
        

    while((step+7) <= len(data)):
    
        for tmp in range(i,step):
            
            X.append(data[tmp])
        
       
        for tmp in range(step,step+step_y):
            
            Y.append(data[tmp])
            
        #Aqui um ciclo acaba
        dataset.append([X,Y])
        X = []
        Y = []
        
        i = i + 1
        step = step + 1
    
    #Agora colocar esta list em numpy array
   
    X = np.array([case[0] for case in dataset]).reshape(len(dataset),time_step)
    y = np.array([case[1] for case in dataset]).reshape(len(dataset),step_y)
        
    #Reshape para ser possível passar à LSTM
    X = X.reshape(len(X),len(X[0]),1)
    y = y.reshape(len(y),len(y[0]),1)
    
    return X,y

X,y = makeXy(data,time_step = 7, forecasting = "single")
#dataset

In [ ]:
print(X[0:2])

print("\n-------------------\n")

print(y[0:1])

## Function to check the loss

In [ ]:
def show_history(history):
    print(history.history.keys())

    # summarize history for accuracy
    #plt.plot(history.history['mean_squared_error'])
    #plt.title('model accuracy with 128 units')
    #plt.ylabel('accuracy')
    #plt.xlabel('epoch')
    #plt.show()
    # summarize history for loss
    plt.plot(history.history['loss'],'r')
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.show()

## Model utilizado para forecasting multi step ou single step
### A função necessita dos seguintes argumentos:

1. Forecasting, que representa a previsão da rede, single step ou multi step
2. time_steps, número de dias que vêm em cada elemento do x_train
3. nr_features, representa o número de features, neste caso só temos 1 feature
3. units, representa o número de unidades para serem utilizadas na LSTM

#### Este modelo é constituído por:

    > LSTM -> Dense -> Dense

In [ ]:
def build_model(forecasting,time_steps,nr_features,units):
    
    #tf.keras.backend.clear_session()
    model = tf.keras.Sequential()
    
    model.add(tf.keras.layers.LSTM(units = units, input_shape= (time_steps, nr_features), return_sequences = False))
    
    if(forecasting == "multi"):
    
        #model.add(tf.keras.layers.LSTM(units = units, return_sequences = False))
    
        model.add(tf.keras.layers.Dense(64,activation='linear'))
        
        model.add(tf.keras.layers.Dense(time_steps,activation='linear'))    
        #model.add(tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(1,activation='linear')))

    if(forecasting == "single"):
        
        #model.add(tf.keras.layers.LSTM(units = units, return_sequences = False))
    
        model.add(tf.keras.layers.Dense(64,activation='linear'))
        
        model.add(tf.keras.layers.Dense(1,activation='linear'))        
        
    opt = tf.keras.optimizers.Adam()
    
    model.compile(optimizer="adam",loss = "mse")# metrics=[tf.keras.metrics.MeanSquaredError()])
    
    return model


from keras.callbacks.callbacks import EarlyStopping

def fit(epochs,model,X,y):
    
    callbacks = [
        EarlyStopping(
            monitor='loss',
            patience=3,
            verbose=2,
            mode='min'
        )
    ]
    
    history = model.fit(X,y,shuffle=False,verbose=2,epochs=epochs)#callbacks=callbacks)
    
    return history

### Multi step model

In [ ]:
X,y = makeXy(data,7,"multi")
model_multi_step  = build_model("multi",7, 1, 128)
model_multi_step.summary()
history = fit(300,model_multi_step,X,y)

In [ ]:
show_history(history)

### Single Step model

In [ ]:
X,y = makeXy(data,7,"single")
model_single_step  = build_model("single",7, 1, 128)
model_single_step.summary()
history = fit(100,model_single_step,X,y)

In [ ]:
show_history(history)

> Esta função faz a previsão nos dados de treino.
> Faço isto para conseguir avaliar visualmente como o modelo se comporta com os dados de treino, ver se está a dar overfitting ou consegue generalizar

In [ ]:
def predict_data(data,model,forecasting,time_step):
    m = len(data)
    i = 0
    predicted_array = []
    
    #dou sempre append dos primeiros 7 valores, são os únicos que não são previstos
    for i in range(time_step):
        predicted_array.append(data[i])
    
    
    if(forecasting == "multi"):
        
        while(i+time_step < m):
            
            #da 7 valores
            predict = model.predict([[data[i:i+time_step]]])[0]
            
            for value in predict:
    
                predicted_array.append(value)

            i = i + time_step
    if(forecasting == "single"):
        
        while(i+time_step < m):
            
            #da 1 valor
            predict = model.predict([[data[i:i+time_step]]])[0]
            
            predicted_array.append(predict[0])

            i = i + 1
            
    
    return np.asarray(predicted_array)

## Esta função devolve todos os resultados referentes aos modelos

1. Devolve a previsão nos dados de treino
2. Devolve a previsão nos 7 dias seguintes


> Model1 representa o model multi step

> Model2 representa o model single step

In [ ]:
def return_results(time_step,model1,model2):

    # Multi Step
    time_step = time_step

    predicted_x_train_multi = predict_data(data,model1,"multi",time_step) #preve nos dados de treino
    predicted_days_multi = model1.predict([[data[-time_step:]]])[0]#preve os 7 dias seguintes


    #Single Step
    predicted_x_train_single = predict_data(data,model2,"single",time_step) #preve nos dados de treino
    predicted_days_single = np.zeros(shape=(time_step,1))

    tmp = 0
    for i in data[-time_step:]:
        predicted_days_single[tmp][0] = i
        tmp = tmp + 1
    

    #preve os proximos 7 dias
    for i in range(time_step):
        value = model2.predict([[predicted_days_single]])[0]#preve os 7 dias seguintes
   
        for i in range(0,len(predicted_days_single)-1):
        
            predicted_days_single[i] = predicted_days_single[i+1]
        
        
        predicted_days_single[i+1] = value
        
    dt1 =  []
    dt2 = []

    for i in data:
        dt1.append(i[0])
        dt2.append(i[0])
    for i in predicted_days_multi:
        dt1.append(i)
    for i in predicted_days_single:
        dt2.append(i[0])
        
    return predicted_x_train_multi,dt1,predicted_x_train_single,dt2

predicted_x_train_multi,predicted_days_multi,predicted_x_train_single, predicted_days_single = return_results(7,model_multi_step,model_single_step)

## Some plots to check if's everything ok

#### Como foi referido, foram utilizados dois modelos, um modelo de multi step e outro modelo single step
#### Ambos estes modelos conseguem convergir e os resultados são apresentados na figura abaixo

1. A reta a preta representa a previsão do modelo single step no x_train, é como se fosse uma métrica de avaliação, mas visual
2. A reta vermelha representa a previsão do modelo multi step no x_train

> Decidi fazer o plot destas duas previsões para ver a diferença do comportamento dos dois modelos nos dados de treino, consegue-se observar que ambos os modelos conseguem generalizar, sendo que o modelo single step apresenta uma variação menos reduzida

3. A reta a azul mostra a previsão nos próximos 7 dias utilizando o modelo multi step
4. A reta azul clara mostra a previsão nos próximos 7 dias utilizando o modelo single step

5. A reta verda representa o número de casos confirmados relativamento ao covid-19

> O que consigo concluir neste exercísio é que modelos com multi-step muito altos apresentam um mau desempanho comparados com single-steps predictions, dado esta previsão tão elevado no multi-step

> Modelos com time-step menor conseguem generalizar melhor, mas normalmente o single step consegue ter previsões mais credíveis.

In [ ]:
from matplotlib.pyplot import figure

plt.figure(figsize=(15,10))
    
plt.plot(predicted_x_train_single,'k')
plt.plot(predicted_x_train_multi,'r')
plt.plot(predicted_days_multi,'b')
plt.plot(predicted_days_single,'c')
plt.plot(data,'g')

plt.title("Number of Confirmed cases with Covid-19")
plt.xlabel("Days")
plt.ylabel("Covid-19")
plt.legend(['Predicted on the same data using forecasting single step',
            'Predicted on the same data using forecasting multi step(7)',
            'Predicted on 7 days using multi step',
            'Predicted on 7 days using single step',
            'Real Data'],loc = 'upper left')
plt.legend
plt.show()

In [ ]:
actual_value = scaler.inverse_transform(np.asarray(data[-1:]).reshape(1,-1))[0][0]
max_value1 = scaler.inverse_transform(np.asarray(predicted_days_single[-1:]).reshape(1,-1))[0][0]
max_value2 = scaler.inverse_transform(np.asarray(predicted_days_multi[-1:]).reshape(1,-1))[0][0]


print("Valor atual de casos confirmados = " + str(actual_value))
print("Valor atual de casos 7 dias depois segundo o model single step = " + str(max_value1))
print("Valor atual de casos 7 dias depois segundo o model multi step = " + str(max_value2))


# Mudando o número de time_Steps consegue-se observar uma diferença nos modelos

In [ ]:
X,y = makeXy(data,4,"multi")
model_ms  = build_model("multi",4, 1, 128)
model_ms.summary()
history = fit(100,model_ms,X,y)

X,y = makeXy(data,4,"single")
model_ss  = build_model("single",4, 1, 128)
model_ss.summary()
history = fit(100,model_ss,X,y)

In [ ]:
predicted_x_train_multi,predicted_days_multi,predicted_x_train_single,predicted_days_single = return_results(4,model_ms,model_ss)

### Com um time step de 4 consegue-se observar um diferença na previsão
### Visualmente qualquer um dos modelos parece correto

> Apesar de que o modelo single step continua a apresentar sempre uma maior variância

In [ ]:
from matplotlib.pyplot import figure

plt.figure(figsize=(15,10))
    
plt.plot(predicted_x_train_single,'k')
plt.plot(predicted_x_train_multi,'r')
plt.plot(predicted_days_multi,'b')
plt.plot(predicted_days_single,'c')
plt.plot(data,'g')

plt.title("Number of Confirmed cases with Covid-19")
plt.xlabel("Days")
plt.ylabel("Covid-19")
plt.legend(['Predicted on the same data using forecasting single step',
            'Predicted on the same data using forecasting multi step(7)',
            'Predicted on 7 days using multi step',
            'Predicted on 7 days using single step',
            'Real Data'],loc = 'upper left')
plt.legend
plt.show()